In [ ]:
#Importamos las librerias
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import time
from numpy import linalg as LA
import random
from time import time
%matplotlib qt

In [10]:
#Definimos la funcion a minimizar
def f(x):
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 -(20*x[0]+40*x[1])
    return fx

In [11]:
#Definir funcion para calcular la matriz del gradiente
def gradient(x,delta):
    grad=np.zeros(2)
    grad[0]=(f([x[0]+delta,x[1]])- f([x[0]-delta,x[1]]))/(2*delta)
    grad[1]=(f([x[0],x[1]+delta])- f([x[0],x[1]-delta]))/(2*delta)
    return grad

In [12]:
#Definir funcion para calcular la matriz Hessiana
def Hessian(x,delta):
    H=np.zeros([2,2])
    H[0,0]= (f([x[0]+delta,x[1]]) - 2*f([x[0],x[1]]) +  f([x[0]-delta,x[1]]))/(delta**2)
    H[1,1]= (f([x[0],x[1]+delta]) - 2*f([x[0],x[1]]) +  f([x[0],x[1]-delta]))/(delta**2)
    H[0,1]= (f([x[0]+delta,x[1]+delta]) - f([x[0]+delta,x[1]-delta]) - f([x[0]-delta,x[1]+delta]) +  f([x[0]-delta,x[1]-delta]))/(4*delta**2)
    H[1,0]= H[0,1]
    return H

In [13]:
#Utilizamos el algoritmo de golden section para encontrar alpha optimo
def golden(x,search,xi,eps):
    a = xi[0];
    b = xi[1];
    tau = 0.381967;
    alpha1 = a*(1-tau) + b*tau;
    alpha2 = a*tau + b*(1-tau);
    falpha1 = f(x+alpha1*search);
    falpha2 = f(x+alpha2*search);
    for i in range(100):
        if falpha1 > falpha2:
            a = alpha1;
            alpha1 = alpha2;
            falpha1 = falpha2;
            alpha2 = tau*a + (1-tau)*b;
            falpha2 = f(x+alpha2*search);
        else:
            b = alpha2;
            alpha2 = alpha1;
            falpha2 = falpha1;
            alpha1 = tau*b + (1-tau)*a;
            falpha1 = f(x+alpha1*search);
        if np.abs(f(x+alpha1*search)- f(x+alpha2*search)) < eps :
            break;
    return alpha1,falpha1

In [14]:
#Graficamos la funcion de prueba
X1 = np.arange(-5, 5, 0.01)
X2 = np.arange(-5, 5, 0.01)
x1, x2 = np.meshgrid(X1, X2)
z = f([x1,x2])
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
surf = ax.plot_surface(x1, x2, z, cmap=cm.autumn, linewidth=0, antialiased = False, alpha=0.2)
plt.show()

In [15]:
#Variable en comun de los métodos
delta = 1e-3 
ep1 = 1e-3
puntosN = []

In [16]:
def graficaUnaaUna(puntos, nombreMetodo):
    #Generamos valores ramdom para generar un color diferente de grafica
    r = random.random()
    b = random.random()
    g = random.random()
    color = (r, g, b)
    aux = 0
    x = np.linspace(-1, 1, 100)
    y = np.linspace(-1, 1, 100)
    xx, yy = np.meshgrid(x, y)
    z = f([xx, yy])
    plt.contour(x,y,z,20)
    plt.plot(-1,-1, 'ro--',  markersize=6)
    plt.plot(0.5,0, 'go--', markersize=6)
    #Graficamos los trazos (direcciones) que va generando el algoritmo
    for pts in puntos:
        if(aux==len(puntos)-1):
            plt.plot(pts[0],pts[1], c=color, linewidth=2, label=str(nombreMetodo))
        else:
            plt.plot(pts[0],pts[1], c=color, linewidth=2)
        aux = aux + 1
    plt.legend()    
    plt.show

In [18]:
start_time = time() #Conteo para tiempo
iteraciones = 0 #Conteo de numero de iteraciones
xi = [-1,1] #Punto de partida
x = xi #Punto inicial para el metodo de golden section
xAnt = xi #Definimos una variable de punto anterior
fx_prev = f(x) #Se evalua la funcion con el punto inicial
print('Valor de la función inicial = %f ' % fx_prev)
for i in range(1000): #Proceso iterativo del algoritmo
    xAnt = x #Almacenamos el punto anterior
    dire = gradient(x,delta) #Se obtiene la matriz del gradiente de la funcion objetivo
    H = Hessian(x,delta) #Se calcula la matriz Hessiana
    dire = np.atleast_2d(dire) #La función se usa cuando queremos convertir entradas en matrices con al menos dos dimensiones. Las entradas escalares y unidimensionales se convierten en matrices bidimensionales, mientras que las entradas de mayor dimensión se conservan.
    si=np.matmul(-LA.inv(H),dire.transpose()) #Se multiplica la inversa de la matriz Hessina por el vector gradiente (de la direccion)
    si = si.transpose() #Ahora la nueva direccion de busqueda es la transpuesta esto porque la multiplicacion
    si = np.ndarray.flatten(si) #Devuelve una copia de la matriz colapsada en una dimensión.
    alpha, fx_curr = golden(x,si,xi,ep1) #Se encuentra el alpha optimo (tamaño de paso)
    if abs(fx_curr-fx_prev)<ep1 or LA.norm(dire)<ep1: #Establecemos como condicion de paro, que la diferencia fx_anterior-fx_actual sea menor a epsilon o que la norma del gradiente sea menor a epsilon
        print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, x[0], x[1],fx_curr,LA.norm(dire)))
        break;
    print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, x[0], x[1],fx_curr,LA.norm(dire)))
    fx_prev=fx_curr
    x = x +  alpha*si #ahora x anterior es x_opt
    puntosN.append([[xAnt[0],x[0]],[xAnt[1],x[1]]])
    iteraciones += 1
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
#Graficamos el resultado del Steepest descent method
graficaUnaaUna(puntosN, 'Modified Newton’s method') 

Valor de la función inicial = 270.294169 
0	[-1.0000,1.0000]		6.5243		444.3158
1	[-0.3266,0.0340]		-3.2287		31.1194
2	[0.1506,0.1848]		-6.8860		34.9350
3	[0.3772,0.0722]		-7.3383		9.3801
4	[0.4844,0.0138]		-7.3515		2.3801
5	[0.4948,0.0042]		-7.3521		0.7266
Tiempo transcurrido: 2.993584
Iteraciones: 5
